<div style="background-color: #f5f7fa; border-left: 6px solid #0077cc; padding: 1.5em; font-family: 'Segoe UI', sans-serif; border-radius: 8px;">
  <h2 style="color: #0077cc;">Configuraciones de Spark</p>
</div>

<div style="background-color: #f5f7fa; border-left: 6px solid #0077cc; padding: 1.5em; font-family: 'Segoe UI', sans-serif; border-radius: 8px;">
  <h2 style="color: #0077cc;">💡 Cargar configuración de Spark desde JSON</h2>
  <p>Podés mantener tus configuraciones separadas del código usando un archivo <code>.json</code>. Esto es útil para:</p>
  <ul>
    <li>✅ Separar lógica del entorno</li>
    <li>✅ Definir distintos ambientes (dev, prod, test)</li>
    <li>✅ Evitar hardcodear parámetros</li>
  </ul>
</div>

In [ ]:
from pyspark import SparkConf
import json

conf = SparkConf()

file_path = "/content/sample_data/spark_configs.json"

with open(file_path) as f:
  spark_configs = json.load(f)

conf.setAll(spark_configs.items())

<div style="background-color: #f0f8ff; border-left: 6px solid #1e90ff; padding: 1.5em; font-family: 'Segoe UI', sans-serif; border-radius: 8px;">
  <h2 style="color: #1e90ff;">🔹 ¿Qué es una SparkSession?</h2>
  <p>Es el <strong>punto de entrada principal</strong> para trabajar con PySpark.</p>
  <ul>
    <li>Permite crear y manipular DataFrames</li>
    <li>Ejecutar SQL directamente sobre los datos</li>
    <li>Leer archivos CSV, JSON, Parquet, entre otros</li>
    <li>Acceder a configuraciones internas</li>
  </ul>
  <h3>📌 Creación básica:</h3>
  <pre style="background-color: #eef6fb; padding: 1em; border-radius: 6px;">
spark = SparkSession.builder \\
    .appName("DataArlaSparkApp") \\
    .master("local[*]") \\
    .getOrCreate()
  </pre>


In [ ]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .config(conf=conf)
    .getOrCreate()
)

🧠 ¿Qué es un DataFrame en PySpark?

In [ ]:
from pyspark.sql.types import *
import random

schema = StructType([
    StructField("id", IntegerType(), False),
    StructField("nombre", StringType(), True),
    StructField("edad", IntegerType(), True)
])

names = ["nacho","facu","ana","julia","jorge"]
data = [(id, random.choice(names), random.randint(16,55)) for id in range(1,11)]

df = spark.createDataFrame(data, schema = schema)

#### Lazy Evaluation

In [ ]:
from pyspark.sql.functions import col
menores_edad = df.filter(col("edad") < 18) # transformacion

In [ ]:
menores_edad.show() # acción

+---+------+----+
| id|nombre|edad|
+---+------+----+
|  1| jorge|  30|
|  6| jorge|  30|
| 10|  facu|  36|
+---+------+----+



In [ ]:
from pyspark.sql.functions import *
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
base_path = '/content/drive/MyDrive/Curso PySpark/datasets for yt'

users_path = f'{base_path}/users.csv'
tickets_path  = f'{base_path}/tickets.csv'
ticket_lines_path =f'{base_path}/ticket_lines.csv'

users_dataframe = spark.read.csv(users_path, header=True)
users_dataframe.printSchema()

<h2>🧠 Selección de columnas con <code>select()</code> y <code>selectExpr()</code></h2>

<p>En PySpark, seleccionamos columnas de un DataFrame para analizarlas o transformarlas. Hay dos métodos muy comunes para hacerlo:</p>

<h3>🔹 <code>select()</code></h3>
<p>Este método permite seleccionar columnas directamente o construir nuevas columnas con funciones.</p>

</code></pre>

<p>✔️ Pros:</p>
<ul>
  <li>Más seguro en ambientes donde se evita el uso de expresiones SQL</li>
  <li>Facilita el uso de funciones</li>
</ul>

<h3>🔸 <code>selectExpr()</code></h3>
<p>Permite utilizar expresiones SQL directamente como strings. Ideal para cálculos rápidos o renombrar columnas.</p>


<p>✔️ Pros:</p>
<ul>
  <li>Muy flexible y potente con expresiones SQL</li>
  <li>Ideal para hacer cálculos y transformaciones rápidas</li>
</ul>

<p>⚠️ Diferencia clave: <code>selectExpr()</code> recibe strings con expresiones SQL, mientras que <code>select()</code> trabaja con objetos de columna.</p>


In [ ]:
users_data_dataframe = (
    users_dataframe.select(
        col("id").cast(IntegerType())
        ,col("gender").alias("genero").cast(IntegerType())
        ,col("birth_year").alias("fecha_nacimiento").cast(IntegerType())
    )
)


In [ ]:
users_data_dataframe_expr = users_dataframe.selectExpr(
    "CAST(id AS INT) AS id",
     "CAST(gender AS INT) AS genero",
     "CAST(birth_year AS INT) AS fecha_nacimiento"
    ).printSchema()

<div style="font-family: Arial, sans-serif; line-height: 1.6">
  <h2>🧱 Transformaciones: <code>withColumn</code> y <code>withColumnRenamed</code></h2>

  <h3>📌 ¿Qué es <code>withColumn</code>?</h3>
  <p>
    El método <code>withColumn(nombre_columna, expresión)</code> permite:
    <ul>
      <li>Agregar una nueva columna calculada al DataFrame.</li>
      <li>Reemplazar una columna existente si el nombre coincide.</li>
    </ul>
    La expresión debe ser un objeto <code>pyspark.sql.Column</code>. Puede contener cálculos, condiciones, valores fijos, etc.
  </p>


  <h3>🔁 ¿Qué es <code>withColumnRenamed</code>?</h3>
  <p>
    El método <code>withColumnRenamed(nombre_antiguo, nombre_nuevo)</code> permite cambiar el nombre de una columna. Devuelve un nuevo DataFrame con la columna renombrada.
  </p>

</div>


In [ ]:
users_dataframe_data = users_dataframe.withColumn("mayor_edad", year(current_date()) - col("birth_year").cast(IntegerType()) >= 18)

In [ ]:
users_dataframe_data.withColumnRenamed("birth_year","fecha_nacimiento").show()